# 본인 데이터베이스에 맞게 수정해주세요
# company 테이블 번호 순서 동일해야 작동해요

In [ ]:
# mysql - vehicles 테이블

# create table vehicles (
#   	vehicle_id int auto_increment primary key,
# 	vehicle_name varchar(45) not null,
# 	vehicle_img text,
#     vehicle_link text,
#     company_id int,
# 	foreign key (company_id) references company(company_id)
# );

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

def crawl_vehicle_data(vehicle_number):
    # 데이터 수집을 위한 리스트 및 집합 초기화
    data_list = []
    model_names_set = set()

    # 웹 드라이버 설정
    wd = webdriver.Chrome()

    # 데이터 수집
    for i in range(2022, 2025):
        for j in range(1, 13):
            if i == 2024 and j == 9:
                break

            print(i, j)
            wd.get(f'https://auto.danawa.com/auto/?Work=record&Tab=Grand&Brand={vehicle_number}&Month={i}-{j:02}-00&MonthTo=')
            time.sleep(2)
            
            try:
                element = wd.find_element(By.CSS_SELECTOR, '#autodanawa_gridC > div.gridMain > article > main > div > table.recordTable.model > tbody')
                rows = element.find_elements(By.TAG_NAME, 'tr')
    
                for row in rows:
                    class_attribute = row.get_attribute('class')
                    if class_attribute and ('sub' in class_attribute or 'model_' in class_attribute):
                        continue  
    
                    model_element = row.find_element(By.CSS_SELECTOR, 'td.title a')
                    vehicle_name = model_element.text.strip()
    
                    if vehicle_name in model_names_set:
                        continue
    
                    link_element = row.find_element(By.TAG_NAME, 'a')
                    vehicle_link = link_element.get_attribute('href')
                    img_element = link_element.find_element(By.TAG_NAME, 'img')
                    vehicle_img = img_element.get_attribute('src')
    
                    data_dict = {
                        '모델명': vehicle_name,
                        '모델 링크': vehicle_link,
                        '모델 이미지': vehicle_img
                    }
                    data_list.append(data_dict)
                    model_names_set.add(vehicle_name)
            except:
                print(f"{vehicle_number}, {i}, {j}해당 요소를 찾을 수 없습니다. 다음으로 넘어갑니다.")
                continue 

    wd.quit()  # 드라이버 종료
    return data_list


In [13]:
import mysql.connector

def save_to_database(data_list, vehicle_id):
    # 데이터베이스 연결 설정
    db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='1234',
        database='pjt01'
    )

    cursor = db.cursor()
    
    # 데이터베이스에 데이터 삽입

    for data in data_list:
        cursor.execute(
            "INSERT INTO vehicles (vehicle_name, vehicle_link, vehicle_img, company_id) VALUES (%s, %s, %s, %s)",
            (data['모델명'], data['모델 링크'], data['모델 이미지'], vehicle_id)
        )

    # 변경사항 저장
    db.commit()

    # 자원 정리
    cursor.close()
    db.close()


In [14]:
# 기아, 현대, 제네시스, KGM, 르노코리아, BMW, 벤츠, 아우디, 폭스바겐, 볼보, 테슬라, 미니, 포르쉐, 렉서스, 토요타
company_numbers = [307, 303, 304, 326, 321, 362, 349, 371, 376, 459, 611, 367, 312, 381, 486, 491]
# company_numbers = {number: index + 1 for index, number in enumerate(company_numbers)}
# {307: 1, 303: 2, 304: 3, 326: 4, 321: 5, 362: 6, 349: 7, 371: 8, 376: 9, 459: 10, 611: 11, 367: 12, 312: 13, 381: 14, 486: 15, 491: 16}

In [16]:
if __name__ == "__main__":
    for i in range(len(company_numbers)):
        vehicle_data = crawl_vehicle_data(company_numbers[i])  # 웹 크롤링
        save_to_database(vehicle_data, i+1)  # 데이터베이스 저장
        print(f'{i+1}번 클리어')

2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2022 8
2022 9
2022 10
2022 11
2022 12
2023 1
2023 2
2023 3
2023 4
2023 5
2023 6
2023 7
2023 8
2023 9
2023 10
2023 11
2023 12
2024 1
2024 2
2024 3
2024 4
2024 5
2024 6
2024 7
2024 8
1번 클리어
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2022 8
2022 9
2022 10
2022 11
2022 12
2023 1
2023 2
2023 3
2023 4
2023 5
2023 6
2023 7
2023 8
2023 9
2023 10
2023 11
2023 12
2024 1
2024 2
2024 3
2024 4
2024 5
2024 6
2024 7
2024 8
2번 클리어
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2022 8
2022 9
2022 10
2022 11
2022 12
2023 1
2023 2
2023 3
2023 4
2023 5
2023 6
2023 7
2023 8
2023 9
2023 10
2023 11
2023 12
2024 1
2024 2
2024 3
2024 4
2024 5
2024 6
2024 7
2024 8
3번 클리어
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
2022 8
2022 9
2022 10
2022 11
2022 12
2023 1
2023 2
2023 3
2023 4
2023 5
2023 6
2023 7
2023 8
2023 9
2023 10
2023 11
2023 12
2024 1
2024 2
2024 3
2024 4
2024 5
2024 6
2024 7
2024 8
4번 클리어
2022 1
2022 2
2022 3
2022 4
2022 5
2022 6
2022 7
202